# EDA

## Import data and set up Weights and Biases to track code

In [1]:
import wandb
import pandas as pd
import os

os.environ["WANDB_NOTEBOOK_NAME"] = "EDA.ipynb"
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: seraphdev. Use `wandb login --relogin` to force relogin


In [2]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,3,0,NaN,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,3,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,3,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,2,8,2019-06-23,0.52,2,8


## Attempt to use `pandas_profiling`

In [3]:
# import pandas_profiling

# profile = pandas_profiling.ProfileReport(df)
# profile.to_widgets()

## Apparently `pandas_profiling` has been deprecated. So let's do this the old fashioned way

In [4]:
df.shape

(20000, 16)

In [5]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,2.000000e+04,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,15877.000000,20000.000000,20000.000000
mean,1.892380e+07,6.746034e+07,40.728455,-73.952125,153.269050,6.992100,23.274100,1.377446,6.955450,112.901200
std,1.101223e+07,7.857936e+07,0.054755,0.046559,243.325609,21.645449,44.927793,1.683006,32.433831,131.762226
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.393540e+06,7.853718e+06,40.689420,-73.983030,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.952117e+07,3.111431e+07,40.722730,-73.955640,105.000000,2.000000,5.000000,0.720000,1.000000,44.000000
75%,2.912936e+07,1.068426e+08,40.762990,-73.936380,175.000000,5.000000,23.000000,2.010000,2.000000,229.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,10000.000000,1250.000000,607.000000,27.950000,327.000000,365.000000


## There seems to be quite a few otliers, so let's take it one column at a time

In [6]:
def find_min_max(s: pd.Series):
    q3 = s.quantile(0.75)
    q1 = s.quantile(0.25)
    iqr = q3 - q1
    bottom = q1 - 1.5 * iqr
    top = q3 + 1.5 * iqr
    return bottom, top

for column in ["price", "minimum_nights"]:
    print(column + ":\n\tmin: %.2f\n\tmax: %.2f" % find_min_max(df[column]))

price:
	min: -90.00
	max: 334.00
minimum_nights:
	min: -5.00
	max: 11.00


### it seems most of the outliers are on the higher end of the dataset, so lets use common sense to filter lower half of the data and round to the nearest $50 mark for price

#### This leaves us with a range of $10 - $350 for price (AirBnB has a minimum price of $10 as found by a google search) and a range of 1 - 11 minimum nights which we will round up to 14 for 2 weeks

In [7]:
price_filter = df["price"].between(10,350)
min_nights_filter = df["minimum_nights"].between(1,14)

new_df = df[price_filter & min_nights_filter].copy()
new_df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.674700e+04,1.674700e+04,16747.000000,16747.000000,16747.000000,16747.000000,16747.000000,13842.000000,16747.000000,16747.000000
mean,1.873316e+07,6.600128e+07,40.727028,-73.949359,121.082642,2.860572,25.856392,1.469779,3.105153,97.520571
std,1.090390e+07,7.770122e+07,0.056214,0.047920,70.664779,2.291717,47.305260,1.734220,20.105021,124.838908
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,10.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.287530e+06,7.748101e+06,40.687595,-73.981230,67.000000,1.000000,1.000000,0.210000,1.000000,0.000000
50%,1.931783e+07,3.052348e+07,40.720070,-73.953650,100.000000,2.000000,7.000000,0.830000,1.000000,25.000000
75%,2.861656e+07,1.019804e+08,40.762980,-73.932665,159.000000,3.000000,27.000000,2.180000,2.000000,179.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,350.000000,14.000000,607.000000,27.950000,327.000000,365.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20000 non-null  int64  
 1   name                            19993 non-null  object 
 2   host_id                         20000 non-null  int64  
 3   host_name                       19992 non-null  object 
 4   neighbourhood_group             20000 non-null  object 
 5   neighbourhood                   20000 non-null  object 
 6   latitude                        20000 non-null  float64
 7   longitude                       20000 non-null  float64
 8   room_type                       20000 non-null  object 
 9   price                           20000 non-null  int64  
 10  minimum_nights                  20000 non-null  int64  
 11  number_of_reviews               20000 non-null  int64  
 12  last_review                     

### Additionally `last_review` appearns to be a date, but is read as a string. Let's encode that as a datetype

In [9]:
new_df["last_review"] = pd.to_datetime(new_df["last_review"])
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16747 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              16747 non-null  int64         
 1   name                            16740 non-null  object        
 2   host_id                         16747 non-null  int64         
 3   host_name                       16740 non-null  object        
 4   neighbourhood_group             16747 non-null  object        
 5   neighbourhood                   16747 non-null  object        
 6   latitude                        16747 non-null  float64       
 7   longitude                       16747 non-null  float64       
 8   room_type                       16747 non-null  object        
 9   price                           16747 non-null  int64         
 10  minimum_nights                  16747 non-null  int64         
 11  nu

## That looks much better now just to close the run and move on to the next step

In [10]:
run.finish()